In [0]:
# Importe de bibliotecas
import pandas as pd


In [0]:
# Carrega os dados
df_01 = pd.read_csv("data/credit_card_parte1.csv", sep=",")
df_02 = pd.read_csv("data/credit_card_parte2.csv", sep=",")

# Unifica os dois DataFrames
df_full = pd.concat([df_01, df_02], axis=0)
print(df_01.shape[0])
print(df_02.shape[0])
print(df_full.shape[0])

In [0]:
# Convert columns V1 - V28 to float64, replacing comma with dot
for i in range(1, 29):
    col = 'V' + str(i)
    df_full[col] = df_full[col].str.replace(',', '.').astype('float64')

In [0]:
display(df_full)